In [1]:
!pip install gensim==3.8.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.2/24.2 MB 21.7 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: gensim
    Found existing installation: gensim 4.0.1
    Uninstalling gensim-4.0.1:
      Successfully uninstalled gensim-4.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scattertext 0.1.7 requires gensim>=4.0.0, but you have gensim 3.8.3 which is incompatible.


In [2]:
import pandas as pd
from nltk.tokenize import sent_tokenize
from wordcloud import WordCloud, ImageColorGenerator
from gensim.summarization.textcleaner import split_sentences
import nltk
import string
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
import transformers
import pandas as pd
from datasets import Dataset
import datasets
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Load data

In [3]:
train_set = pd.read_excel("../input/summerization/Competition CERIST -- Summarization/Dataset 2 Arabic + English (XL_sum)/English/dataset_XL_sum_v1.0_train_en.xlsx")
test_set = pd.read_excel("../input/summerization/Competition CERIST -- Summarization/Dataset 2 Arabic + English (XL_sum)/English/dataset_XL_sum_v1.0_test_en.xlsx")
train_ds = Dataset.from_pandas(train_set)
test_ds = Dataset.from_pandas(test_set)

In [4]:
train_dataset, validation_dataset= train_ds.train_test_split(test_size=0.1).values()
data_all_splits = datasets.DatasetDict({"train":train_dataset,"test":test_ds, "val":validation_dataset})

# EDA

## some statistics 

In [5]:
def sent_count_text(example):
    example["sent_count_document"]= len(split_sentences(example['Document']))
    return example

def sent_count_summary(example):
    example["sent_count_summary"]= len(split_sentences(example['Summary']))
    return example


In [6]:
data_all_splits=data_all_splits.map(sent_count_text)
data_all_splits=data_all_splits.map(sent_count_summary)

  0%|          | 0/274291 [00:00<?, ?ex/s]

  0%|          | 0/11529 [00:00<?, ?ex/s]

  0%|          | 0/30477 [00:00<?, ?ex/s]

  0%|          | 0/274291 [00:00<?, ?ex/s]

  0%|          | 0/11529 [00:00<?, ?ex/s]

  0%|          | 0/30477 [00:00<?, ?ex/s]

In [7]:
data_all_splits

DatasetDict({
    train: Dataset({
        features: ['Document', 'Summary', 'sent_count_document', 'sent_count_summary'],
        num_rows: 274291
    })
    test: Dataset({
        features: ['Document', 'Summary', 'sent_count_document', 'sent_count_summary'],
        num_rows: 11529
    })
    val: Dataset({
        features: ['Document', 'Summary', 'sent_count_document', 'sent_count_summary'],
        num_rows: 30477
    })
})

In [8]:

def word_count_text(example):
    example['word_count_document'] = len(example['Document'].split())
    return example

def word_count_summary(example):
    example['word_count_summary'] = len(example['Summary'].split())
    return example

In [9]:
data_all_splits=data_all_splits.map(word_count_text)
data_all_splits=data_all_splits.map(word_count_summary)

  0%|          | 0/274291 [00:00<?, ?ex/s]

  0%|          | 0/11529 [00:00<?, ?ex/s]

  0%|          | 0/30477 [00:00<?, ?ex/s]

  0%|          | 0/274291 [00:00<?, ?ex/s]

  0%|          | 0/11529 [00:00<?, ?ex/s]

  0%|          | 0/30477 [00:00<?, ?ex/s]

In [10]:
def char_count_document(example):
    example['char_count_document']= len(example['Document'].replace(" ",""))
    return example
    
def char_count_summary(example):
    example['char_count_summary']= len(example['Summary'].replace(" ",""))
    return example

In [11]:
data_all_splits=data_all_splits.map(char_count_document)
data_all_splits=data_all_splits.map(char_count_summary)

  0%|          | 0/274291 [00:00<?, ?ex/s]

  0%|          | 0/11529 [00:00<?, ?ex/s]

  0%|          | 0/30477 [00:00<?, ?ex/s]

  0%|          | 0/274291 [00:00<?, ?ex/s]

  0%|          | 0/11529 [00:00<?, ?ex/s]

  0%|          | 0/30477 [00:00<?, ?ex/s]

In [12]:
def word_density_summary(example):
    example['word_density_summary'] = example['word_count_summary'] / (example['char_count_summary'] + 1)
    return example
    
    
def word_density_document(example):
    example['word_density_document'] = example['word_count_document'] / (example['char_count_document'] + 1)
    return example

In [13]:
data_all_splits.map(word_density_summary)
data_all_splits.map(word_density_document)

  0%|          | 0/274291 [00:00<?, ?ex/s]

  0%|          | 0/11529 [00:00<?, ?ex/s]

  0%|          | 0/30477 [00:00<?, ?ex/s]

  0%|          | 0/274291 [00:00<?, ?ex/s]

  0%|          | 0/11529 [00:00<?, ?ex/s]

  0%|          | 0/30477 [00:00<?, ?ex/s]

DatasetDict({
    train: Dataset({
        features: ['Document', 'Summary', 'sent_count_document', 'sent_count_summary', 'word_count_document', 'word_count_summary', 'char_count_document', 'char_count_summary', 'word_density_document'],
        num_rows: 274291
    })
    test: Dataset({
        features: ['Document', 'Summary', 'sent_count_document', 'sent_count_summary', 'word_count_document', 'word_count_summary', 'char_count_document', 'char_count_summary', 'word_density_document'],
        num_rows: 11529
    })
    val: Dataset({
        features: ['Document', 'Summary', 'sent_count_document', 'sent_count_summary', 'word_count_document', 'word_count_summary', 'char_count_document', 'char_count_summary', 'word_density_document'],
        num_rows: 30477
    })
})

In [14]:
def sent_density_summary(example):
    example['sent_density_summary'] = example['sent_count_summary'] / (example['word_count_summary'] + 1)
    return example

def sent_density_document(example):
    example['sent_density_document'] = example['sent_count_document'] / (example['word_count_document'] + 1)
    return example

In [15]:
data_all_splits.map(sent_density_summary)
data_all_splits.map(sent_density_document)

  0%|          | 0/274291 [00:00<?, ?ex/s]

  0%|          | 0/11529 [00:00<?, ?ex/s]

  0%|          | 0/30477 [00:00<?, ?ex/s]

  0%|          | 0/274291 [00:00<?, ?ex/s]

  0%|          | 0/11529 [00:00<?, ?ex/s]

  0%|          | 0/30477 [00:00<?, ?ex/s]

DatasetDict({
    train: Dataset({
        features: ['Document', 'Summary', 'sent_count_document', 'sent_count_summary', 'word_count_document', 'word_count_summary', 'char_count_document', 'char_count_summary', 'sent_density_document'],
        num_rows: 274291
    })
    test: Dataset({
        features: ['Document', 'Summary', 'sent_count_document', 'sent_count_summary', 'word_count_document', 'word_count_summary', 'char_count_document', 'char_count_summary', 'sent_density_document'],
        num_rows: 11529
    })
    val: Dataset({
        features: ['Document', 'Summary', 'sent_count_document', 'sent_count_summary', 'word_count_document', 'word_count_summary', 'char_count_document', 'char_count_summary', 'sent_density_document'],
        num_rows: 30477
    })
})

In [16]:
def count_stopwords_document(example):
    ''' Return the number of stopwords in the text
        Input:
            - text: string
            - stopwords: list of string, containing the stopwords
        Output:
            - int, number of stopwords in the text argument
    '''
    word_tokens =nltk.word_tokenize(example['Document'])
    stopwords_x = [w for w in word_tokens if w in  set(stopwords.words('english'))]
    example['count_stopwords_summary'] = len(stopwords_x)
    return example

def count_stopwords_summary(example):
    ''' Return the number of stopwords in the summary
        Input:
            - text: string
            - stopwords: list of string, containing the stopwords
        Output:
            - int, number of stopwords in the text argument
    '''
    word_tokens =nltk.word_tokenize(example['Summary'])
    stopwords_x = [w for w in word_tokens if w in  set(stopwords.words('english'))]
    example['count_stopwords_summary'] = len(stopwords_x)
    return example

In [17]:
#data_all_splits.map(count_stopwords_summary)
#data_all_splits.map(count_stopwords_document)

In [18]:
def ponctuation_count_summary(example):
    example['ponctuation_count_summary'] =  len([a for a in example['Summary'] if a in string.punctuation])
    return example

def ponctuation_count_document(example):
    example['ponctuation_count_document'] =  len([a for a in example['Document'] if a in string.punctuation])
    return example

In [19]:
data_all_splits.map(ponctuation_count_summary)
data_all_splits.map(ponctuation_count_document)

  0%|          | 0/274291 [00:00<?, ?ex/s]

  0%|          | 0/11529 [00:00<?, ?ex/s]

  0%|          | 0/30477 [00:00<?, ?ex/s]

  0%|          | 0/274291 [00:00<?, ?ex/s]

  0%|          | 0/11529 [00:00<?, ?ex/s]

  0%|          | 0/30477 [00:00<?, ?ex/s]

DatasetDict({
    train: Dataset({
        features: ['Document', 'Summary', 'sent_count_document', 'sent_count_summary', 'word_count_document', 'word_count_summary', 'char_count_document', 'char_count_summary', 'ponctuation_count_document'],
        num_rows: 274291
    })
    test: Dataset({
        features: ['Document', 'Summary', 'sent_count_document', 'sent_count_summary', 'word_count_document', 'word_count_summary', 'char_count_document', 'char_count_summary', 'ponctuation_count_document'],
        num_rows: 11529
    })
    val: Dataset({
        features: ['Document', 'Summary', 'sent_count_document', 'sent_count_summary', 'word_count_document', 'word_count_summary', 'char_count_document', 'char_count_summary', 'ponctuation_count_document'],
        num_rows: 30477
    })
})

We can see that train and test are **similar** except for the max characters.

In [20]:
import re
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('omw-1.4')
def clean_text(text):
    
    text = text['Document'].lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"who's", "who is ", text)
    text = re.sub(r"she's", "she is ", text)
    text = re.sub(r"it's", "it is ", text)


    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "can not ",text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ",text)
    text = re.sub(r"\'re", " are ",text)
    text = re.sub(r"\'d", " would ",text)
    text = re.sub(r"\'ll", " will ", text)
    # to remove punctuation
    text.translate(str.maketrans('', '', string.punctuation))

    text = re.sub('\W', ' ', text)
    text= re.sub('\s+', ' ', text)
    text = text.strip(' ')
    ## stemming and lemmatization
    lemmatizer = WordNetLemmatizer()
    for i in range(len(text)):
        words=nltk.word_tokenize(text)
        # remove stop words and apply lemmatization
        words = [lemmatizer.lemmatize(word) for word in words if word not in set(stopwords.words('english'))]
        text = ' '.join(words)
    example['clean_Document']
    return example

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...


In [22]:
test = data_all_splits['test'].to_pandas()
val = data_all_splits['val'].to_pandas()
train = data_all_splits['train'].to_pandas()

In [23]:
train.describe()

,sent_count_document,sent_count_summary,word_count_document,word_count_summary,char_count_document,char_count_summary
count,274291.000000,274291.000000,274291.000000,274291.000000,274291.000000,274291.000000
mean,20.425858,1.078938,459.740837,22.219147,2263.935623,110.148372
std,16.074259,0.348352,336.194600,7.221011,1624.341329,35.014546
min,1.000000,1.000000,32.000000,5.000000,176.000000,40.000000
25%,11.000000,1.000000,240.000000,18.000000,1192.000000,89.000000
50%,16.000000,1.000000,366.000000,21.000000,1814.000000,107.000000
75%,26.000000,1.000000,603.000000,25.000000,2981.000000,122.000000
max,529.000000,11.000000,6366.000000,172.000000,27723.000000,797.000000


In [24]:
val.describe()

,sent_count_document,sent_count_summary,word_count_document,word_count_summary,char_count_document,char_count_summary
count,30477.000000,30477.000000,30477.000000,30477.000000,30477.000000,30477.000000
mean,20.329691,1.078945,459.155133,22.225646,2261.315648,110.254487
std,15.382686,0.343315,328.276245,7.312905,1591.117921,35.463314
min,1.000000,1.000000,35.000000,6.000000,169.000000,40.000000
25%,11.000000,1.000000,241.000000,18.000000,1198.000000,89.000000
50%,16.000000,1.000000,367.000000,21.000000,1818.000000,108.000000
75%,26.000000,1.000000,604.000000,25.000000,2987.000000,122.000000
max,323.000000,7.000000,6139.000000,122.000000,28130.000000,630.000000


In [25]:
test.describe()

,sent_count_document,sent_count_summary,word_count_document,word_count_summary,char_count_document,char_count_summary
count,11529.000000,11529.000000,11529.000000,11529.000000,11529.000000,11529.000000
mean,19.218666,1.035736,437.198630,21.241651,2158.596669,105.640298
std,11.957447,0.203474,262.373004,4.753051,1279.351023,22.933391
min,2.000000,1.000000,61.000000,8.000000,290.000000,43.000000
25%,11.000000,1.000000,246.000000,18.000000,1221.000000,89.000000
50%,16.000000,1.000000,360.000000,21.000000,1783.000000,106.000000
75%,24.000000,1.000000,574.000000,24.000000,2828.000000,121.000000
max,113.000000,5.000000,1667.000000,42.000000,8344.000000,220.000000
